In [10]:
import pandas as pd

In [11]:
df=pd.read_csv('dataset/ronb_dataset.csv',delimiter=';')

In [12]:
df.head()

,Description,Intent
0,Flood/Landslide Update: Death count due to cur...,Sad
1,Kathmandu bata Pokhara jadai gareko Car Trisul...,Sad
2,Growing danger in Kathmandu area: A boy of aro...,Sad
3,Kathmandu District Administration Office le ma...,Surprise
4,Popular Netflix Series Money Heist ( La Casa d...,Surprise


In [13]:
df.shape

(625, 2)

In [14]:
df.tail()

,Description,Intent
620,Medical Students protesting in front of Kathma...,Sad
621,CIAA (Akhtiyar) le Krishi Samagri Company Ltd ...,Surprise
622,Swikar Poudel from Pokhara got 3rd position in...,Surprise
623,Current fiscal year ko first 5 months ma Nepal...,Sad
624,Chauchau ko Carton bhitra NRs. 50 Lakh Cash lu...,Funny


In [15]:
df.columns

Index(['Description', 'Intent'], dtype='object')

In [16]:
df['Intent'].value_counts()

Surprise    280
Sad         241
Funny        72
Angry        32
Name: Intent, dtype: int64

In [17]:
df['Intent'].unique()

array(['Sad', 'Surprise', 'Funny', 'Angry'], dtype=object)

In [18]:
df[df['Intent']=='nan']

,Description,Intent


In [19]:
df[df['Intent'].isna()]

,Description,Intent


In [20]:
df.iloc[14].values

array(["A discussion about Nepal's current situation happened at UK's parliament where the members talked about helping Nepal in present condition. They also talk about how Nepal's Gurkhas have helped UK in need. P.S. UK is sending medical team next week to help Nepal.",
       'Surprise'], dtype=object)

In [70]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit

from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder

In [71]:
tgt = df['Intent']

In [72]:
np.unique(tgt)

array(['Angry', 'Funny', 'Sad', 'Surprise'], dtype=object)

In [73]:
# copying the dataframe 

t2 = df.copy()

In [74]:
sss = StratifiedShuffleSplit(test_size=0.1, random_state=42, n_splits=1)

In [75]:
for train_index, test_index in sss.split(t2,tgt):
    x_train, x_test = t2[t2.index.isin(train_index)], t2[t2.index.isin(test_index)]
    y_train, y_test = t2.loc[t2.index.isin(train_index),"Intent"], t2.loc[t2.index.isin(test_index),"Intent"]

In [76]:
x_train.shape

(562, 2)

In [77]:
x_test.shape

(63, 2)

In [78]:
def get_max_len(list1):
    '''returns the maximum length of the sentence'''
    len_list = [len(i) for i in list1]
    return max(len_list)

In [79]:
tokenizer = Tokenizer()

In [91]:
def conv_str_cols(col_tr,col_te):
    tokenizer = Tokenizer(num_words=2000)
    tokenizer.fit_on_texts(col_tr)
    col_tr1 = tokenizer.texts_to_sequences(col_tr)
    col_te1 = tokenizer.texts_to_sequences(col_te)
    max_len1 = get_max_len(col_tr1)
    col_tr2 = pad_sequences(col_tr1, maxlen=max_len1, dtype='int32',padding='post')
    col_te2 = pad_sequences(col_te1, maxlen=max_len1, dtype='int32',padding='post')
    
    return col_tr2,col_te2,tokenizer,max_len1
    

In [92]:
tr_padded,te_padded, tokenizer,max_len1 = conv_str_cols(x_train["Description"], x_test["Description"])

In [93]:
tr_padded.shape,te_padded.shape

# 11 is the length of sentence after padding and the former are the number of samples

((562, 157), (63, 157))

In [94]:
max_len1

157

In [95]:
le = LabelEncoder()

y_train1 = le.fit_transform(y_train)
y_test1 = le.fit_transform(y_test)
y_train2 = to_categorical(y_train1)
y_test2 = to_categorical(y_test1)

In [96]:
classes_num = len(y_train.value_counts())

In [97]:
classes_num

4

In [98]:
## You embed the input layers and pass them to LSTM. The output of LSTM gets fed to a time-distributed dense layer. This layer gets fed into a 
# fully connected dense layer with classes_num as the number of nodes.

In [99]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM,Flatten,TimeDistributed

In [100]:
model = Sequential()
model.add(Embedding(2000, 100, input_length=max_len1))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(50, activation='relu')))
model.add(Flatten())
model.add(Dense(classes_num, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 157, 100)          200000    
                                                                 
 lstm_6 (LSTM)               (None, 157, 100)          80400     
                                                                 
 lstm_7 (LSTM)               (None, 157, 50)           30200     
                                                                 
 time_distributed_3 (TimeDis  (None, 157, 50)          2550      
 tributed)                                                       
                                                                 
 flatten_3 (Flatten)         (None, 7850)              0         
                                                                 
 dense_7 (Dense)             (None, 4)                 31404     
                                                      

In [101]:
model.fit(tr_padded, y_train2, epochs=10, verbose=2,batch_size=30)

Epoch 1/10
19/19 - 4s - loss: 1.1721 - accuracy: 0.4342 - 4s/epoch - 231ms/step
Epoch 2/10
19/19 - 2s - loss: 1.0750 - accuracy: 0.5018 - 2s/epoch - 108ms/step
Epoch 3/10
19/19 - 2s - loss: 0.7790 - accuracy: 0.7224 - 2s/epoch - 104ms/step
Epoch 4/10
19/19 - 2s - loss: 0.4276 - accuracy: 0.8149 - 2s/epoch - 104ms/step
Epoch 5/10
19/19 - 2s - loss: 0.2462 - accuracy: 0.9110 - 2s/epoch - 107ms/step
Epoch 6/10
19/19 - 2s - loss: 0.1683 - accuracy: 0.9288 - 2s/epoch - 110ms/step
Epoch 7/10
19/19 - 2s - loss: 0.1140 - accuracy: 0.9466 - 2s/epoch - 104ms/step
Epoch 8/10
19/19 - 2s - loss: 0.1145 - accuracy: 0.9644 - 2s/epoch - 105ms/step
Epoch 9/10
19/19 - 2s - loss: 0.0627 - accuracy: 0.9875 - 2s/epoch - 101ms/step
Epoch 10/10
19/19 - 2s - loss: 0.0376 - accuracy: 0.9893 - 2s/epoch - 100ms/step


### Model Prediction and Evaluation

In [102]:
# pred_mat = model.predict_classes(te_padded)


predict_x=model.predict(te_padded) 
classes_x=np.argmax(predict_x,axis=1)

In [103]:
from sklearn.metrics import accuracy_score

print (accuracy_score(y_test1, classes_x))

0.6666666666666666


In [104]:
from sklearn.metrics import f1_score

print (f1_score(y_test1, classes_x,average='micro'))
print (f1_score(y_test1, classes_x,average='macro'))

0.6666666666666666
0.4818756539937217


In [57]:
def pred_new_text(txt1):
    print ("customer_text:",txt1)
    newdf = pd.DataFrame([txt1])
    newdf.columns = ["Description"]
    col_te1 = tokenizer.texts_to_sequences(newdf["Description"])
    col_te2 = pad_sequences(col_te1, maxlen=max_len1, dtype='int32',padding='post')
#     class_pred = le.inverse_transform(model.predict_classes(col_te2))[0]
    predict_x=model.predict(te_padded) 
    classes_x=le.inverse_transform(np.argmax(predict_x,axis=1))[0]
    return classes_x

In [58]:
pred_new_text('aja kathmandu ma heavy rainfall vayekoley bato ma road accident vayeko xa')

customer_text: aja kathmandu ma heavy rainfall vayekoley bato ma road accident vayeko xa


'Surprise'